In [1]:
import os
import yfinance as yf
import pandas as pd
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver 
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from typing import TypedDict, Annotated, List
import operator
from langchain_core.messages import BaseMessage


@tool
def get_trade_signals(ticker: str):
    """
    Calculates Support (Entry), Resistance (Target), and Stop Loss.
    """
    try:
        stock = yf.Ticker(ticker)
        # Fetch 1 year of data to calculate valid 60-day support
        hist = stock.history(period="1y")
        
        if hist.empty: return "Error: No Data found."

        # --- INDICATORS ---
        # 1. Trend (SMA 50)
        hist['SMA_50'] = hist['Close'].rolling(window=50).mean()
        
        # 2. Support & Resistance (60-day Lookback)
        hist['Support'] = hist['Low'].rolling(window=60).min()
        hist['Resistance'] = hist['High'].rolling(window=60).max()
        
        # 3. Bollinger Bands (Volatility)
        hist['SMA_20'] = hist['Close'].rolling(window=20).mean()
        hist['STD_20'] = hist['Close'].rolling(window=20).std()
        hist['BB_Lower'] = hist['SMA_20'] - (hist['STD_20'] * 2)
        
        # Current Data Points
        curr = hist.iloc[-1]
        price = curr['Close']
        stop_loss = curr['Support'] * 0.97
        
        return (
            f"--- TECHNICAL DATA FOR {ticker} ---\n"
            f"Price: {price:.2f}\n"
            f"Support (Entry): {curr['Support']:.2f}\n"
            f"Resistance (Target): {curr['Resistance']:.2f}\n"
            f"Stop Loss (3% Risk): {stop_loss:.2f}\n"
            f"BB Lower: {curr['BB_Lower']:.2f}\n"
        )
    except Exception as e:
        return f"Error: {e}"

# --- 2. SETUP ENVIRONMENT ---
# Initialize LLM
llm = ChatGroq(model="llama-3.1-70b-versatile", temperature=0)

# Connect to VectorDB
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
try:
    vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embedding)
    print("✅ VectorDB Connected.")
except:
    vectorstore = None
    print("⚠️ Warning: VectorDB not found.")

print("✅ Notebook 3 Setup Complete.")

C:\Users\DELL\AppData\Local\Temp\ipykernel_16916\2722444012.py:63: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embedding)


✅ VectorDB Connected.
✅ Notebook 3 Setup Complete.


In [2]:

print("--- TEST 1: Technical Analysis Tool ---")

print(get_trade_signals.invoke("RELIANCE.NS"))



--- TEST 1: Technical Analysis Tool ---
--- TECHNICAL DATA FOR RELIANCE.NS ---
Price: 1546.30
Support (Entry): 1356.90
Resistance (Target): 1581.30
Stop Loss (3% Risk): 1316.19
BB Lower: 1465.17



In [3]:

print("\n--- TEST 2: Fundamental Retrieval ---")
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

try:
    embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    db = Chroma(persist_directory="./chroma_db", embedding_function=embedding)
    retriever = db.as_retriever(search_kwargs={"k": 2})
    
    docs = retriever.invoke("Risks and challenges")
    if docs:
        print(f"Retrieved: {docs[0].page_content[:200]}...")
    else:
        print("No documents found (Did you add a PDF?)")
except Exception as e:
    print(f"RAG Test Failed: {e}")


--- TEST 2: Fundamental Retrieval ---
Retrieved: workplaces & enhance operational 
efficiency & have site-specific 
emergency plans & conduct regular 
mock drills.
RIL has extensive systems meeting or 
exceeding regulatory requirements 
ensuring saf...
